In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import rnn_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'rnn'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 15
EXECUTIONS_PER_TRIAL = 2

TOP_N = 4
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_rnn_type = hp.Choice("rnn_type", values=["lstm", "gru"])
    hp_n_neurons = hp.Choice("n_neurons", values=[16, 32, 64, 128, 256, 512])
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=0, max_value=1, default=0)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization")
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_rnn_type, hp_n_neurons, hp_n_hidden_layers, hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 lstm (LSTM)                 (None, 16)                1152      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 15 Complete [00h 01m 12s]
val_loss: 9.249314785003662

Best val_loss So Far: 8.705897331237793
Total elapsed time: 00h 32m 00s


# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'rnn_type': 'lstm', 'n_neurons': 256, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 264545
========== Model 1 ==========
{'rnn_type': 'gru', 'n_neurons': 32, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 3489
========== Model 2 ==========
{'rnn_type': 'gru', 'n_neurons': 256, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 199265
========== Model 3 ==========
{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': True}
Number of parameters: 37985


# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'rnn_type': 'lstm', 'n_neurons': 256, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 264545
Got score: 8.7755 (8.7828, 8.7682)
Got score: 8.4416 (8.4581, 8.4251)
Got score: 8.1763 (8.1420, 8.2106)
Got score: 8.5108 (8.5235, 8.4981)
Got score: 8.9612 (8.8247, 9.0977)


{'rnn_type': 'gru', 'n_neurons': 32, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 3489
Got score: 8.7285 (8.7304, 8.7266)
Got score: 8.4371 (8.4776, 8.3966)
Got score: 8.1967 (8.2189, 8.1744)
Got score: 8.5573 (8.5829, 8.5317)
Got score: 8.9841 (8.9882, 8.9800)


{'rnn_type': 'gru', 'n_neurons': 256, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 199265
Got score: 8.8708 (8.7415, 9.0001)
Got score: 8.3383 (8.3650, 8.3117)
Got score: 8.2817 (8.2811, 8.2824)
Got score: 8.8028 (8.5579, 9.0477)
Got score: 8.8928 (8.7998, 8.9858)


{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': True}
Number of parameters: 37985
Got score: 9.1078 (9.0839, 9.1316)
Got score: 8.9098 (8.9529, 8.8667)
Got score: 8.4833 (8.4909, 8.4757)
Got score: 8.7834 (8.7882, 8.7785)
Got score: 9.2915 (9.2869, 9.2961)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.57,0.27,264545
1,8.58,0.27,3489
2,8.64,0.27,199265
3,8.92,0.28,37985
